# Transparency in Coverage Data For Mere Mortals - Part 1

July 1, 2022, the [Center for Medicare & Medicaid Services](https://www.cms.gov/) (CMS) began enforcing its new [Transparency in Coverage Rule](https://www.cms.gov/healthplan-price-transparency), which requires almost all US health plans to post very detailed pricing data about the vast majority of the services and procedures these plans cover.

In this Jupyter Notebook series, I will provide code and commentary that enables mere mortals like you and me to draw insights from the Transparency in Coverage datasets that payers have released. Some of [this data is quite large](https://github.com/CMSgov/price-transparency-guide/discussions/528) and it can seem impossible to wrangle, but stick with me and I'll show you how to get around the huge data issue. We'll go through various approaches of ingesting these datasets using the Open Source programming language [Python](https://www.python.org/) and other Open Source and (eventually) cloud-based tools to efficiently crawl through these files and extract exactly what we need.

1. In **Part 1**, this notebook, I will briefly discuss the Transparency in Coverage data standard, focusing on building Python code that can extract data from the TOC (Table of Contents) file as it is defined by the CMS' standard. To do that, we'll need to learn some JSON basics along the way and then write a little Python to download, parse, and interpret JSON files. I'll walk you through all of that in this notebook.

2. In **Part 2**, we'll dive deeper, and learn about what makes these files so huge - and how to use streaming and decompression to our advantage. We'll discuss the tradeoff between system resources (such as memory and processing power) and time as we design our parser. We'll also learn about [SQLite](https://www.sqlite.org/index.html), a powerful lightweight [RDBMS](https://en.wikipedia.org/wiki/Relational_database) that allows us to write data on disk so we can keep our memory usage low as we extract the data from these truly massive files. At the end of Part 2, we'll have extracted never-before-seen negotiated in-network rates out of some pretty large files!

3. In **Part 3**, we'll continue extracting payer files by turning our attention to providers and provider groups. We'll learn about APIs, and how we can use our new mastery of JSON to combine data we extract from these files with JSON-formatted data we request from the [NPPES NPI API](https://npiregistry.cms.hhs.gov/api-page).

3. In **Part 4**, we'll tackle the biggest of the Transparency in Coverage in-network files out there from some of the big national payers, and we'll show how [freely-available / free-tier cloud resources on Microsoft Azure](https://azure.microsoft.com/en-us/pricing/free-services/) can be used to bridge the gap between single-user computer capabilities and the capabilities afforded to us by a whole cloud arsenal at our fingertips!

*(The targeted audience for this notebook series is actuaries who have some familiarity with the Python lagnuage, but you can probably also learn as you go if you've not seen much Python before.)*

## Transparency in Coverage Schema
The [CMS has published a guide on Github](https://github.com/CMSgov/price-transparency-guide/) that defines how the data required for compliance with the new Transparency in Coverage Rule should be packaged by payers, in accordance with [85 FR 72158](https://www.federalregister.gov/documents/2019/11/27/2019-25011/transparency-in-coverage).

Our first step will be to select an arbitrary "Table of Contents" (hereafter, "TOC") file from a payer to use as an example. The [TOC file schema](https://github.com/CMSgov/price-transparency-guide/tree/master/schemas/table-of-contents) is described here. 

Payer data is required to be stored without any "gatekeeping" of any kind. Therefore, in most cases, you can [Google Search](https://www.google.com/search?q=transparency+in+coverage+machine+readable+carefirst&sxsrf=ALiCzsbF3J4fo2uJfS8HNcU6iwmNNC-ikA%3A1661350949782&ei=JTQGY5KfL_GYptQP-ZivwAM&ved=0ahUKEwjSxdXJ1t_5AhVxjIkEHXnMCzgQ4dUDCA4&uact=5&oq=transparency+in+coverage+machine+readable+carefirst&gs_lcp=Cgdnd3Mtd2l6EAMyBQgAEKIEMgUIABCiBDIFCAAQogQ6BwgAEEcQsAM6BwgjELACECc6CAgAEB4QCBANOgUIABCGAzoHCAAQHhCiBDoECCEQCkoECEEYAEoECEYYAFDbCljSFWCZLWgFcAB4AIABZYgB4wqSAQQxOC4xmAEAoAEByAEIwAEB&sclient=gws-wiz) your way to the sections of large payer websites where this data is kept.

Some payers have elected to mask downloadable links through JavaScript wrappers. It is uncertain whether or not this practice will be tolerated as compliant by the CMS in the future.

In order to make sure I'm not promoting the total spamming of a particular single health carrier's hosting services by releasing this tutorial, you need to supply your very own link to a health insurer's TOC file in the code cell below.

I have provided a list below of some options for TOC files. Please navigate to one of the sites below and obtain a URL for a Table of Contents file from a carrier into the code cell below. Then copy that URL into the cell below. (**Note**: For now, make sure the TOC file you decide to use is *not* compressed. This is something we will learn how to deal with later in this notebook, and also cover in more detail in Part 2. The [CMS has specified in response to an issue raised on Github](https://github.com/CMSgov/price-transparency-guide/discussions/59#discussioncomment-642881) that compression is allowed for any machine-readable file published under these rules. I have specified which carriers below are publishing their TOC files in compressed formats, so please steer clear of those for now.)

- [Blue Cross Blue Shield North Carolina](https://www.bluecrossnc.com/about-us/policies-and-best-practices/transparency-coverage-mrf)
- [Cigna](https://www.cigna.com/legal/compliance/machine-readable-files)
- [CareFirst Blue Cross Blue Shield (Maryland)](https://individual.carefirst.com/individuals-families/mandates-policies/machine-readable-file.page)
- [CVS fka Aetna](https://health1.aetna.com/app/public/#/one/machine-readable-transparency-in-coverage) **NOTE:** Aetna's TOC files are compressed, so you will need to download them locally and uncompress them first. We will actually learn how to do this later in this notebook, but for now, I recommend not using Aetna's TOC files.
- [Elevance fka Anthem](https://www.anthem.com/machine-readable-file/search/)
- [Humana](https://developers.humana.com/Cost-Transparency) **NOTE:** Unfortunately, Humana has chosen to go off script vs. the official CMS standard for reporting this data, so you'll have to do quite a bit of code editing in the Python below to make Humana work for you. I recommend not using Humana's TOC files.
- [UnitedHealth Group](https://transparency-in-coverage.uhc.com/?_gl=1*13tpnkt*_ga*MTA1ODQ5NjQ4MC4xNjYwODY2MDg0*_ga_HZQWR2GYM4*MTY2MTY1MTMxMi4yLjAuMTY2MTY1MTMxMi4wLjAuMA..) **NOTE:** UHG's TOC files are compressed, so you will need to download them locally and uncompress them first. We will actually learn how to do this later in this notebook, but for now, I recommend not using UHG's TOC files.
- [HCSC](https://www.hcsc.com/who-we-are/regulatory-disclosures) (BCBS licensee for Texas, Illinois, New Mexico, Montana, and Oklahoma).
- [Independence Blue Cross](https://www.ibx.com/cmstic/?brand=qcc) (BCBS licensee for southeastern PA area)
- [Horizon Blue Cross](https://horizonblue.sapphiremrfhub.com/) (BCBS licensee for New Jersey)
- [Empire Blue Cross](https://www.empireblue.com/machine-readable-file/search/) (BCBS licensee for New York) **NOTE:** This one is giant, and also compressed. Avoid using this one for now.
- [Blue Cross Blue Shield of Arizona](https://www.ahatpa.com/transparency-in-coverage/2019093?key=H8vC/RthBP8lvbpPlvzmbK/QiOnuQkvJOu2WY/poNibNeP+FlA2f5DCqcRSmVhwvDaHibcHqygWQC/ZlmhUOhHhD)
- [Blue Cross Blue Shield of California](https://web.healthsparq.com/healthsparq/public/#/one/insurerCode=BSCA_I&brandCode=BSCA/machine-readable-transparency-in-coverage)
- [Blue Cross Blue Shield South Carolina](https://www.southcarolinablues.com/web/public/brands/sc/about-us/privacy-and-legal/transparency-in-coverage/)
- [Amerihealth](https://www.amerihealth.com/cmstic/?brand=ahpa)
- [Highmark and many other Blues as published by enGen](https://mrfdata.hmhs.com/)

In [73]:
import requests

# Paste the Table of Contents file URL that you obtained in the previous step in this string here:
#URL = "https://mrfmftprod.bcbsnc.com/prod/etl/outbound/table-of-contents/aso/2022-07-27_sas-institute-inc_index.json"
#URL = "https://storage.googleapis.com/ihg-dart-edw-mrf-prod-public/qcc/2022-08-01_qcc_index.json"
#URL = "https://mrf.carefirst.com/mrf-files/Table-Of-Content-Carefirst-HMO.json"
URL = "https://storage.googleapis.com/ihg-dart-edw-mrf-prod-public/qcc/2022-08-01_qcc_index.json"

#URL = ""

response = requests.get(URL)
open("toc_file.json", "wb").write(response.content)

780345

## A quick JSON primer

### What is JSON?

[JSON](https://en.wikipedia.org/wiki/JSON) is a format that is used to add structure to data. It stands for "**J**ava**S**cript **O**bject **N**otation," and it is a very popular way to convey data over the internet. Almost all of the services you use every day, such as streaming services, home comfort control services, and generally anything with a public (or private!) API all likely use JSON to transmit and receive data with the consumer devices and interfaces you use to control these services. It is a format that is very human-readable and exceptionally machine-readable, and it offers more versatility in the way data is stored over traditional "flat" methods, such a CSV file.

YouTube is rife with great videos about JSON basics, and I'd encourage you to take a moment to watch a few. It won't take more than 10 or 15 minutes to fully grasp the big idea. I personally found [this video](https://www.youtube.com/watch?v=nlYiOcMNzyQ) to be very helpful without being *too* technical - but if you want something more technical, you've definitely got options out there.

### Bring Your Own Schema

In the video I linked above, the author mentioned that **JSON does not provide for any one specific schema**. What does this mean, exactly? JSON provides a syntax or common grammar/language for adding complex structure to data, but exactly *what* that structure is, as well as how that structure is defined and interpreted, is left up to the programmers who are building the programming API or tool. Let's take a look at an example.

#### FHIR and JSON

If you are a healthcare actuary, you might have heard that [CMS has designated the FHIR HL7 standard as the required means of health data communication between providers, patients, and payers over which they have regulatory authority](https://www.federalregister.gov/documents/2020/12/18/2020-27593/medicaid-program-patient-protection-and-affordable-care-act-reducing-provider-and-patient-burden-by). In this case, [FHIR HL7](https://build.fhir.org/documentation.html) is the schema or standard - but the language used to structure this schema is often JSON.

| FHIR Patient Data in Human Readable Diagram      | FHIR Patient Data in JSON  | 
| :----:      |    :----:   | 
| ![A screenshot of the FHIR HL7 patient object in a human-readable diagram](./images/FHIR_patient_structure.png "FHIR HL7 Patient Data Diagram")      | ![A screenshot of the FHIR HL7 patient object in JSON](./images/FHIR_patient_JSON.png "FHIR HL7 Patient Data in JSON")       | 

Above is an example from [the official FHIR HL7 documentation for patient data](https://build.fhir.org/patient.html). On the left, you'll see an image of a very human-readable diagram that shows the how data about a patient is structured heirarchically in a way that makes a lot of sense. This is the data's *schema*, its structure. This has little to do with JSON. Where JSON comes in is on the right: JSON is one particular format we can use to express the data schema so that a machine can easily, quickly, and consistently read the data.

Suppose you are a payer and you need to make a prior authorization decision on a provider hospital's request to admit a patient into the ER. As part of this request, the provider will send health information to the payer that is relevant to the decision. In the past, this was done in many ways - via fax, over email, using proprietary forms and schemas, etc. The big idea of Interoperability is that all the data anyone could want is structured in the FHIR HL7 schema. And that data can be efficiently broken down, transmitted, and re-assembled in a human-readable way if it's structured in JSON format, among other options.

## Reading the Table of Contents File

Per the CMS Standard, the first level of the TOC Schema should consist of elements that contain basic information about the info we're reading in. Let's start with getting that info. If this code doesn't work, then you should check the code in the previous code cell above to make sure that the URL you selected points to a valid JSON file.

In [74]:
import ijson
import pandas as pd

toc_file = open('./toc_file.json')

base_toc = [(prefix, event, value) for (prefix, event, value) in ijson.parse(toc_file) if prefix == '']

print('\n'.join('{}: {}'.format(*k) for k in enumerate(base_toc)))

toc_file.close()

0: ('', 'start_map', None)
1: ('', 'map_key', 'reporting_entity_name')
2: ('', 'map_key', 'reporting_entity_type')
3: ('', 'map_key', 'reporting_structure')
4: ('', 'end_map', None)


When you run this code, you should see something like:

    0: ('', 'start_map', None)
    1: ('', 'map_key', 'reporting_entity_name')
    2: ('', 'map_key', 'reporting_entity_type')
    3: ('', 'map_key', 'reporting_structure')
    4: ('', 'end_map', None)

If you *don't* see this, you should go and pick another TOC URL from the list above, and run that in the first cell, or double check that your first URL is an actual JSON file.

In the code cell above, note the line:

`base_toc = [(prefix, event, value) for (prefix, event, value) in ijson.parse(toc_file) if prefix == '']`

Here, we are using a special feature of Python (and some other languages) known as [List Comprehension](https://en.wikipedia.org/wiki/List_comprehension). This is a way of building a list containing a subset of information drawn from elements of a larger set. In plain language, what this means is:

1. We'd like to call `ijson.parse` on our entire `toc_file`, which is an open connection to our `toc_file.json` we downloaded in the previous section.
2. From this method, we want all the 3-tuple combinations of `(prefix, event, value)` that `ijson.parse` has returned.
3. We want to subset this further such that the list only contains 3-tuple combinations where there is no value for the prefix event - meaning that we're looking at the root-level JSON elements for this file.
4. The square brackets `[]` around this code indicate we want the subset of 3-tuples to be expressed as a list, one that is assigned to the variable `base_toc`.

We can easily compare the output to here to the base-level elements we're expecting to see, as described by the [CMS published standard](https://github.com/CMSgov/price-transparency-guide/tree/master/schemas/table-of-contents) for this data.

![An inline image screenshot of the CMS published standard for table of contents files.](./images/CMS_TOC_Standard.png "CMS TOC root-level elements")


So, we're on the right track! Let's press on. Since `reporting_entity_name` and `reporting_entity_type` should have character string values, pull those out.

In [75]:
toc_file = open('./toc_file.json')
reporting_entity_name = [(prefix, event, value) for (prefix, event, value) in ijson.parse(toc_file) if prefix == 'reporting_entity_name']
print('\n'.join('{}: {}'.format(*k) for k in enumerate(reporting_entity_name)))
toc_file.close()

toc_file = open('./toc_file.json')
reporting_entity_type = [(prefix, event, value) for (prefix, event, value) in ijson.parse(toc_file) if prefix == 'reporting_entity_type']
print('\n'.join('{}: {}'.format(*k) for k in enumerate(reporting_entity_type)))
toc_file.close()


0: ('reporting_entity_name', 'string', 'QCC Insurance Company')
0: ('reporting_entity_type', 'string', 'health insurance issuer')


The above cell's code should output something like:

    0: ('reporting_entity_name', 'string', 'ABC HEALTH INSURANCE CORP')
    0: ('reporting_entity_type', 'string', 'HEALTH INSURANCE ISSUER')

Hopefully your output is as expected. Also note that there could be multiple entities reported in a single TOC file, but most TOC files out there so far just have one entity to report.

Now let's peel the onion back a bit. According to the CMS standard, the `reporting_structure` item that is a root member of the TOC file is itself a JSON array ("\[a\]n array of reporting structure object types" per the standard), which we can express as a Python list. So let's investigate and see if the `reporting_structure` array is what we're expecting, which is this:

![An inline image screenshot of the CMS published standard for reporting_structure_objects.](./images/CMS_Reporting_Structure_Object.png "CMS Reporting Structure Object root-level elements")


In [76]:
toc_file = open('./toc_file.json')

reporting_structure_objects = [(prefix, event, value) for (prefix, event, value) in ijson.parse(toc_file) if prefix == 'reporting_structure.item']
print('\n'.join('{}: {}'.format(*k) for k in enumerate(reporting_structure_objects)))

toc_file.close()


0: ('reporting_structure.item', 'start_map', None)
1: ('reporting_structure.item', 'map_key', 'reporting_plans')
2: ('reporting_structure.item', 'map_key', 'in_network_files')
3: ('reporting_structure.item', 'map_key', 'allowed_amount_file')
4: ('reporting_structure.item', 'end_map', None)


If the TOC file is compliant with the CMS standard, you should see something like:

    0: ('reporting_structure.item', 'start_map', None)
    1: ('reporting_structure.item', 'map_key', 'reporting_plans')
    2: ('reporting_structure.item', 'map_key', 'in_network_files')
    3: ('reporting_structure.item', 'map_key', 'allowed_amount_file')
    4: ('reporting_structure.item', 'end_map', None)

You might see this output just one time, or you may see it a number of times repeated. We are expecting at this point to see an array of `reporting_structure` objects. An array is simply a computer science term for a list of items, and so for your TOC file, there might be a list of several `reporting structure` objects, or just one.

If your experience is as expected, we see that there is an array of `reporting_structure` objects, as defined above, and they contain the three objects as expected: `reporting_plans`, `in_network_files`, and `allowed_amount_file`.


## Extracting the Reporting Structure Information

There is nothing in the Reporting Structure Object definition in the CMS standard that contains identifying information about the structure itself: in other words, there is no string value within `reporting_structure` that names the structure. Instead, the six `reporting_structure` objects here are provided, and each one has the following features:

1. `reporting_plans`: an array of Reporting Plan objects
2. `in_network_files`: an array of File Location objects associated with each item in `reporting_plans`
3. `allowed_amount_files`: a single File Location object associated with each item in `reporting_plans`

Knowing this up front makes it very easy for us to start scraping and organizing this data. Let's begin!

### Reporting Plans Array

We know this is an array of Reporting Plans objects. This is what we expect these objects to look like:

![An inline image screenshot of the CMS published standard for reporting_plan objects.](./images/CMS_Reporting_Plans_Object.png "CMS Reporting Plan Object root-level elements")

Note that this is object is the lowest "level" we can peel back, since every data type here is basically a "primitive" data type - which is computer science-speak for a single-valued data type, vs. an array or an object containing other data inside of it. This means that we can make a table that represents the all the Reporting Plans objects, because each item in this array of Reporting Plans objects can be represented as a [Python `dict` type](https://en.wikibooks.org/wiki/Python_Programming/Dictionaries), which can be thought of as an individual row in a data table. Since there are presumably *many* Reporting Plans objects, we can iterate over all of of them with our parser and assemble them into a `pandas` DataFrame. 

In [77]:
toc_file = open('./toc_file.json')

reporting_plans = pd.DataFrame(ijson.items(toc_file, 'reporting_structure.item.reporting_plans.item'))
reporting_plans['index'] = reporting_plans.reset_index().index
display(reporting_plans)

toc_file.close()

,plan_name,plan_id_type,plan_id,plan_market_type,index
0,COMMERCIAL EPO,hios,31609,group,0
1,COMMERCIAL PPO,hios,31609,group,1
2,COMMERCIAL PPO,hios,31609PA007,individual,2
3,COMMERCIAL PPO,hios,31609PA015,group,3
4,COMMERCIAL EPO,hios,31609PA016,individual,4
5,COMMERCIAL PPO,hios,31609PA016,individual,5
6,COMMERCIAL EPO,hios,31609PA017,group,6
7,COMMERCIAL EPO,hios,31609PA018,individual,7
8,COMMERCIAL PPO,hios,31609PA018,individual,8
9,COMMERCIAL PPO,hios,31609PA019,individual,9


This is very encouraging, finally some useful data! It looks like there are many `reporting_plans` objects. But one challenge here is that we don't know how these items relate to their parent objects, `reporting_structure`. We know there is at least one `reporting_structure` object within this TOC file, but all we've done is list all of the `reporting_plans` objects without understanding how each of these plans relate to their parent objects, `reporting_structure`.  We'll want to keep track of what `reporting_structure` each `reporting_plans` object belongs to!

In [79]:
toc_file = open('./toc_file.json')

reporting_plans = ijson.items(toc_file, 'reporting_structure.item.reporting_plans')
rps = pd.DataFrame()
for (struct_num, rep_plan) in enumerate(reporting_plans):
    plan_details = pd.DataFrame([plan for plan in rep_plan])
    plan_details['reporting_structure_number'] = struct_num
    rps = rps.append(plan_details, ignore_index=True)
    
display(rps)

toc_file.close()

,plan_name,plan_id_type,plan_id,plan_market_type,reporting_structure_number
0,COMMERCIAL EPO,hios,31609,group,0
1,COMMERCIAL PPO,hios,31609,group,0
2,COMMERCIAL PPO,hios,31609PA007,individual,0
3,COMMERCIAL PPO,hios,31609PA015,group,0
4,COMMERCIAL EPO,hios,31609PA016,individual,0
5,COMMERCIAL PPO,hios,31609PA016,individual,0
6,COMMERCIAL EPO,hios,31609PA017,group,0
7,COMMERCIAL EPO,hios,31609PA018,individual,0
8,COMMERCIAL PPO,hios,31609PA018,individual,0
9,COMMERCIAL PPO,hios,31609PA019,individual,0


### Quick Knowledge Check

Use the table output from the above code to answer the following questions?

1. How many `reporting_structure` objects are there in the TOC file? Remember - Python starts counting at 0, not 1.
2. How many `reporting_plans` objects are there in the TOC file?

Use the code below to check your answers!


In [82]:
# Number of reporting_structure objects:
print("The number of reporting_structure objects is: {}".format(rps['reporting_structure_number'].max()+1))

# Number of reporting_plans objects:
print("The number of reporting_structure objects is: {:,}".format(max(rps.index+1)))

The number of reporting_structure objects is: 1
The number of reporting_structure objects is: 10


### In-Network Files Array

Here again we have to be careful - just reading in all the items under "map_start" events in all items marked "in_network_files" gets us a giant list of `file_location` type Objects. And we will want to have a way to relate these back to the `reporting_structure` or `reporting_plans` objects we've already discovered.

In [83]:
toc_file = open('./toc_file.json')

in_network_files = pd.DataFrame(ijson.items(toc_file, 'reporting_structure.item.in_network_files.item'))
in_network_files['index'] = in_network_files.reset_index().index
display(in_network_files)

toc_file.close()

,description,location,index
0,in-network file,https://independencebc.mrf.bcbs.com/2022-08_02...,0
1,in-network file,https://independencebc.mrf.bcbs.com/2022-08_02...,1
2,in-network file,https://independencebc.mrf.bcbs.com/2022-08_03...,2
3,in-network file,https://independencebc.mrf.bcbs.com/2022-08_04...,3
4,in-network file,https://independencebc.mrf.bcbs.com/2022-08_04...,4
...,...,...,...
1472,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,1472
1473,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,1473
1474,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,1474
1475,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,1475


Here, we see that there are many `file_location` objects in the `in_network_file` array, but it might be the case that there are not the same number of `reporting_plans` - so we will need to take care to group the `in_network_file` array items alongside the `reporting_structure` items to which they belong, just as we did earlier with `reporting_plans`. We might also want to see a readout of the number of `file_location` objects belong to each `reporting_structure`.

In [84]:
toc_file = open('./toc_file.json')

in_network_files = ijson.items(toc_file, 'reporting_structure.item.in_network_files')
infs = pd.DataFrame()
for (struct_num, in_netw_f) in enumerate(in_network_files):
    in_net_f_detail = pd.DataFrame([inf for inf in in_netw_f])
    in_net_f_detail['reporting_structure_number'] = struct_num
    infs = infs.append(in_net_f_detail, ignore_index=True)
    
display(infs)
display(pd.DataFrame(infs.value_counts('reporting_structure_number')))

toc_file.close()

,description,location,reporting_structure_number
0,in-network file,https://independencebc.mrf.bcbs.com/2022-08_02...,0
1,in-network file,https://independencebc.mrf.bcbs.com/2022-08_02...,0
2,in-network file,https://independencebc.mrf.bcbs.com/2022-08_03...,0
3,in-network file,https://independencebc.mrf.bcbs.com/2022-08_04...,0
4,in-network file,https://independencebc.mrf.bcbs.com/2022-08_04...,0
...,...,...,...
1472,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,0
1473,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,0
1474,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,0
1475,in-network file,https://mrfdata.hmhs.com/files/362/ibc/inbound...,0


,0
reporting_structure_number,
0,1477


Perfect, we now have collected all the `in_network_file` items, and furthermore have kept track of which `reporting_structure` they belong to. Note that some `reporting_structures` might use the same set of `file_location` objects.

### Allowed Amount File Object

As we can see in the `reporting_structure` Object definition above, the `allowed_amount_file` item should refer to only one `file_location` object and not an array of them, as did the `in_network_files` item. This should make collecting the associated URL for the Allowed Amount File very easy - and we should expect only six of them.

What *is* the allowed amount data going to contain, anyways?

![An inline image screenshot of requirements concerning what allowed amount values to include in the data taken from the CMS reporting standard github site.](./images/CMS_Allowed_Amt_Additional_Note.png "CMS Allowed Amounts - Additional Note")

### This reporting structure can encourage redundant data

You may have heard people talking about how absolutely massive these Transparency in Coverage datasets are. For example, Cigna's in-network rates files can top several hundred gigabytes, and that's *before* you uncompress them.

Pay attention to that last linein the definition of allowed amounts above - that these variations need to be captured at the plan *or* issuer level! So, conceptually, you could have the same in-network rates for all of your plans - but if your allowed amount files differed at the level of reporting across plans, then that would force you to create a separate `reporting_structure` object in which `in_network_file` objects are completely repeated just to accommodate the different `allowed_amount_file` objects.

In [85]:
toc_file = open('./toc_file.json')

allowed_amt_file = ijson.items(toc_file, 'reporting_structure.item.allowed_amount_file')

aafs = pd.DataFrame([aaf for aaf in allowed_amt_file])    
aafs['reporting_structure_number'] = aafs.index
display(aafs)

toc_file.close()

,description,location,reporting_structure_number
0,allowed amount file,https://storage.googleapis.com/ihg-dart-edw-mr...,0


### Finishing up with a final join? Or how about not...

Let's finish our work here by collecting our `reporting_plans` data table as well as our `in_network_files` data table, knowing that we can join their respective subsets at any time on the `reporting_structure_number` field as the join key for both. While JSON can often be "flattened" into a traditional table-style format with rows and columns, it doesn't always mean that you *should* do this. In most cases, it would be a very large table with many repeated values. Instead, we should store the three primary objects - `reporting_plans`, `in_network_files` and `allowed_amount_file` - as separate tables, as well as any additional metadata information about the TOC file.

Since our goal at the end of this might be to parse this information and warehouse it in a more efficient storage format, it makes sense to be comfortable with having three datasets that can be joined at any time down the line, if required, vs. a much larger dataset.

Just to illustrate the point, here's the size of a table that results from the merge of the `reporting_plans` table with the `in_network_files` table. In most cases, this table will be massively larger than either of the two tables it is created from, so there is just no reason to do this merge.

In [86]:
final_toc_in_netw_out = pd.DataFrame(pd.merge(rps, infs, on=['reporting_structure_number', 'reporting_structure_number'], how="inner", indicator=True).count())

display(final_toc_in_netw_out)

,0
plan_name,14770
plan_id_type,14770
plan_id,14770
plan_market_type,14770
reporting_structure_number,14770
description,14770
location,14770
_merge,14770


## TIC-TOC Helper Function

One of the best parts about having a published standard to which payers are (mostly) adhering is that the Transparency in Coverage (TIC) data always should always be structured the same, no matter what insurer you're getting it from. Thus, all the TOC files in the TIC data should be readable with a generalized function that parses the data in the way we did today.  

So, let's finish up by building a parse_tic_toc function that takes as input an open file connection, and runs the parse operations we've described here, outputting the items in a 3-tuple of objects for us to reference later.

### A Note on GZIP Compression
URLs from these TiC files generally come to us in two forms - .json files which are compressed in GZIP format, so they have an extension of `.json.gz`, and those which are not compressed, which merely have an extension of `.json`. At first, we want our helper function to have the ability to parse both kinds once the file is downloaded. 

*(Extra points if we can even skip downloading the file, and instead stream the contents of these URLs over the HTTPS protocol incrementally through our parser function. We're not there yet, but we'll tackle that task in the next TiC Tutorial Notebook.)*

### Why is GZIP Helpful Here?
I want to note that GZIP compression a so-called "sliding window" compression method (please enjoy this [explainer video on the compression algorithm on which gzip is based](https://www.youtube.com/watch?v=goOa3DGezUA)). This means that we can actually parse JSON from within a GZIPped file without uncompressing it entirely on disk first. 

While this might seem like trivial knowledge now, in the next TiC Tutorial notebook we will encounter some of these super-huge in-network rate files that you've heard about. We're going to tackle a few that are fairly large, even if compressed. Many of these files come in sizes that are just impossible to accomodate on most computers. Reinflating compressed GZIP files to their former glory can somtimes mean a file size that is 100 times the size of its compressed form. In this case, 30 GB might become nearly 3,000 GB, or 3 terabytes! This creates challenges for parsing the file, because we'll need to find a way to extract the information we need from these files *without* ingesting them entirely into memory.

### Why is the Compression Ratio So Large?
As I've mentioned before, due to the highly redundant nature of this data, the GZIP compression ratio for these files is quite good, averaging around 95% - 99%. Generally, whenever you have lots of repeated information in a text file, GZIP compression ratios are higher.

### Compressed vs. Uncompressed: ¿Por qué no los dos?
Ultimately, we want to build into our `parse_tic_toc` helper function the capability to parse JSON whether it encounters it in compressed or uncompressed format, so we've designed for that with the initial block of code:

    if file_path.endswith('.json.gz'):
        toc_file = gzip.open('./toc_file_aetna.json.gz', 'rb')
    elif file_path.endswith('.json'):
        toc_file = open('./toc_file.json')
    else:
        raise ValueError("Expecting a .json or .json.gz file...")

In other words, we want our helper method to open a streaming connection to the file that varies based on the file extension - if it's `.json.gz` then we'll open the file with the method `gzip.open()`, if it's `.json` then we'll simply the standard file reader, and if it's neither, we'll raise an error and halt execution.

In [87]:
import gzip

def parse_tic_toc(file_path):
    
    ###### Do a first pass collecting Reporting Entity and Type
    if file_path.endswith('.json.gz'):
        toc_file = gzip.open(file_path, 'rb')
    elif file_path.endswith('.json'):
        toc_file = open(file_path)
    else:
        raise ValueError("Expecting a .json or .json.gz file...")

    parser = ijson.parse(toc_file)
    for prefix, event, value in parser:
        if prefix == 'reporting_entity_name':
            reporting_entity_name = value
        if prefix == 'reporting_entity_type':
            reporting_entity_type = value
    
    toc_file.close()
    
    ###### Next, collect Reporting Plans
    if file_path.endswith('.json.gz'):
        toc_file = gzip.open(file_path, 'rb')
    elif file_path.endswith('.json'):
        toc_file = open(file_path)
    else:
        raise ValueError("Expecting a .json or .json.gz file...")

    reporting_plans = ijson.items(toc_file, 'reporting_structure.item.reporting_plans')
    rps = pd.DataFrame()
    for (struct_num, rep_plan) in enumerate(reporting_plans):
        plan_details = pd.DataFrame([plan for plan in rep_plan])
        plan_details['reporting_structure_number'] = struct_num
        rps = rps.append(plan_details, ignore_index=True)
        
    rps['reporting_entity_name'] = reporting_entity_name
    rps['reporting_entity_type'] = reporting_entity_type
    
    toc_file.close()
    
    ###### Next, collect In-Network File Location Objects
    if file_path.endswith('.json.gz'):
        toc_file = gzip.open(file_path, 'rb')
    elif file_path.endswith('.json'):
        toc_file = open(file_path)
    else:
        raise ValueError("Expecting a .json or .json.gz file...")
        
    in_network_files = ijson.items(toc_file, 'reporting_structure.item.in_network_files')
    infs = pd.DataFrame()
    for (struct_num, in_netw_f) in enumerate(in_network_files):
        in_net_f_detail = pd.DataFrame([inf for inf in in_netw_f])
        in_net_f_detail['reporting_structure_number'] = struct_num
        infs = infs.append(in_net_f_detail, ignore_index=True)
        
    infs['reporting_entity_name'] = reporting_entity_name
    infs['reporting_entity_type'] = reporting_entity_type
    
    toc_file.close()
    
    ###### Finally, collect Allowed Amount File Location Objects
    if file_path.endswith('.json.gz'):
        toc_file = gzip.open(file_path, 'rb')
    elif file_path.endswith('.json'):
        toc_file = open(file_path)
    else:
        raise ValueError("Expecting a .json or .json.gz file...")
        
    allowed_amt_file = ijson.items(toc_file, 'reporting_structure.item.allowed_amount_file')
    aafs = pd.DataFrame([aaf for aaf in allowed_amt_file])    
    aafs['reporting_structure_number'] = aafs.index
    
    aafs['reporting_entity_name'] = reporting_entity_name
    aafs['reporting_entity_type'] = reporting_entity_type
    
    return rps, infs, aafs

### Four Passes?!

As you might have noticed, in our helper method above, we are making four passes over the data. This is a trade-off to consider. Parsing large files takes time, and crawling over a massive file doing string comparisons is time consuming. On the other hand, from a memory management perspective, if we attempt to extract too much data at one time during our pass over the file, we might max out our available memory. We have to balance one constraint (time) with the other (memory).

In the next section, we'll be working with some pretty big files. These files are so big that it might not make a lot of sense to download them to disk. Instead, we can consider streaming them through our JSON parser and collecting only the information we need - which is a much smaller subset of the information in these files. 

Ultimately, some of the rate files we will be working with are *so* large that it's not possible to download them to disk, and it's *also* not possible to extract their contents with the memory and disk constraints of your typical single-user computer, so in Part 3 of this tutorial, we'll be expanding our arsenal of tools into the cloud. But for now, let's be happy with where we're at so far, and build on to our parsing method with incremental improvements, which we'll discuss in Part 2 of this series.

### Testing

Finally, let's test our data on a number of options. 

1. First, on our own TOC file. 
2. Then, on a very large TOC file from Cigna. **Note**: *Cigna has elected to include ALL of its plans in one TOC file, which includes ASO plans for many employer plan sponsors, so this test takes about a minute to run.*
3. Then, on a very large TOC from Aetna that is a compressed `.json.gz` file
4. Finally, on an image (.png) file presented within this notebook, just to make sure our error handling is functioning accordingly.


In [69]:
# Testing - our own TOC File
toc_file = './toc_file.json'
rps, infs, aafs = parse_tic_toc(toc_file)
display(rps, infs, aafs)

# More testing - Cigna - uncomment the next three lines on the first run to download the file
#URL = "https://d25kgz5rikkq4n.cloudfront.net/cost_transparency/mrf/table-of-contents/reporting_month=2022-08/2022-08-01_cigna-health-life-insurance-company_index.json?Expires=1663258504&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9kMjVrZ3o1cmlra3E0bi5jbG91ZGZyb250Lm5ldC9jb3N0X3RyYW5zcGFyZW5jeS9tcmYvdGFibGUtb2YtY29udGVudHMvcmVwb3J0aW5nX21vbnRoPTIwMjItMDgvMjAyMi0wOC0wMV9jaWduYS1oZWFsdGgtbGlmZS1pbnN1cmFuY2UtY29tcGFueV9pbmRleC5qc29uIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjYzMjU4NTA0fX19XX0_&Signature=NucG2ID8F7zsGtIqNirj1uliIPIuFuhEapXIC3MjTN4cjvDwoJiZ0X2-4PRERH7i0Y-T99~xUFBsO~NjkegP4R2HGgcZygAT6C5T6NHl1UY-~qowDIl3KnujEvNJLvxOEYftbZsE7yfpPWXlV8sqM5dvItJrRuQEhP6Du9kBYA~SifOtfLUz-a6wn9QdVbsfPo80mUHqq~OBuk3HOJigBJbS0miiUHRhvEbTdMa9Nu5VSwMLTrod850P~kh~TgzEB4MTP-B-PrarwUCgsu4aYP3Eh2OMSIy4kxnL8xtlhBL7W0EiUUlpvVgsOTUScp43eyGC0Mmi5LMnEwqLD8HJsg__&Key-Pair-Id=K1NVBEPVH9LWJP"
#response = requests.get(URL)
#open("toc_file_cigna.json", "wb").write(response.content)

toc_file = './toc_file_cigna.json'
rps, infs, aafs = parse_tic_toc(toc_file)
display(rps, infs, aafs)

# More testing - Aetna - parsing through a GZIP'd file
# URL = "https://mrf.healthsparq.com/aetnacvs-egress.nophi.kyruushsq.com/prd/mrf/AETNACVS_I/ALICFI/2022-08-05/tableOfContents/2022-08-05_Aetna-Life-insurance-Company_index.json.gz"
# response = requests.get(URL)
# open("toc_file_aetna.json.gz", "wb").write(response.content)

toc_file = './toc_file_aetna.json.gz'
rps, infs, aafs = parse_tic_toc(toc_file)
display(rps, infs, aafs)

# Testing - an image (for which we expect an error)
toc_file = './images/CMS_TOC_Standard.png'
rps, infs, aafs = parse_tic_toc(toc_file) 

,plan_name,plan_id_type,plan_id,plan_market_type,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,BLUECHOICE ADVANTAGE CDH_POS,EIN,01-0550163,group,0,CareFirst Inc,HEALTH INSURANCE ISSUER
1,BLUECHOICE ADVANTAGE CDH_POS,EIN,01-0564360,group,0,CareFirst Inc,HEALTH INSURANCE ISSUER
2,BLUECHOICE ADVANTAGE CDH_POS,EIN,01-0592798,group,0,CareFirst Inc,HEALTH INSURANCE ISSUER
3,BLUECHOICE ADVANTAGE CDH_POS,EIN,01-0827222,group,0,CareFirst Inc,HEALTH INSURANCE ISSUER
4,BLUECHOICE ADVANTAGE CDH_POS,EIN,02-0717122,group,0,CareFirst Inc,HEALTH INSURANCE ISSUER
...,...,...,...,...,...,...,...
6119,TDN_HMO,EIN,00-00099M8,individual,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6120,TDN_HMO,EIN,00-00099M9,individual,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6121,TDN_HMO,EIN,00-00099MA,individual,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6122,TDN_HMO,EIN,00-00099MB,individual,24,CareFirst Inc,HEALTH INSURANCE ISSUER


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
1,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
2,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
3,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
4,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
...,...,...,...,...,...
6796,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6797,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6798,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,24,CareFirst Inc,HEALTH INSURANCE ISSUER
6799,Carefirst in-network HMO file,https://carefirstbcbs.mrf.bcbs.com/2022-08_690...,24,CareFirst Inc,HEALTH INSURANCE ISSUER


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,0,CareFirst Inc,HEALTH INSURANCE ISSUER
1,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,1,CareFirst Inc,HEALTH INSURANCE ISSUER
2,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,2,CareFirst Inc,HEALTH INSURANCE ISSUER
3,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,3,CareFirst Inc,HEALTH INSURANCE ISSUER
4,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,4,CareFirst Inc,HEALTH INSURANCE ISSUER
5,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,5,CareFirst Inc,HEALTH INSURANCE ISSUER
6,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,6,CareFirst Inc,HEALTH INSURANCE ISSUER
7,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,7,CareFirst Inc,HEALTH INSURANCE ISSUER
8,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,8,CareFirst Inc,HEALTH INSURANCE ISSUER
9,Carefirst allowed amount HMO file,https://mrf.carefirst.com/mrf-files/allowed-am...,9,CareFirst Inc,HEALTH INSURANCE ISSUER


,plan_name,plan_id_type,plan_id,plan_market_type,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,"NATIONAL OAP COMPETITION CAMS, INC.",ein,621009281,group,0,Cigna Health Life Insurance Company,Health Insurance Issuer
1,NATIONAL OAP CIGNA Health and Life Insurance C...,ein,59-1031071,group,1,Cigna Health Life Insurance Company,Health Insurance Issuer
2,LOCALPLUS Weld County Garage,ein,840348620,group,2,Cigna Health Life Insurance Company,Health Insurance Issuer
3,SAN FRANCISCO SACRAMENTO HMO CIGNA HEALTHCARE ...,ein,95-3310115,group,3,Cigna Health Life Insurance Company,Health Insurance Issuer
4,METRO NEW YORK GPPO CIGNA Health and Life Insu...,ein,59-1031071,group,4,Cigna Health Life Insurance Company,Health Insurance Issuer
...,...,...,...,...,...,...,...
34235,"NATIONAL OAP Sunshine Media Group, Inc.",ein,651064897,group,18233,Cigna Health Life Insurance Company,Health Insurance Issuer
34236,"LOCALPLUS C3 Industries, Inc.",ein,832247786,group,18234,Cigna Health Life Insurance Company,Health Insurance Issuer
34237,NATIONAL OAP CM & Associates Construction Mana...,ein,204915944,group,18235,Cigna Health Life Insurance Company,Health Insurance Issuer
34238,"NATIONAL OAP InterConnect Wiring, LLP",ein,752459439,group,18236,Cigna Health Life Insurance Company,Health Insurance Issuer


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,Cigna Health Life Insurance Company,Health Insurance Issuer
1,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,1,Cigna Health Life Insurance Company,Health Insurance Issuer
2,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2,Cigna Health Life Insurance Company,Health Insurance Issuer
3,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,3,Cigna Health Life Insurance Company,Health Insurance Issuer
4,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,4,Cigna Health Life Insurance Company,Health Insurance Issuer
...,...,...,...,...,...
2435,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2435,Cigna Health Life Insurance Company,Health Insurance Issuer
2436,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2436,Cigna Health Life Insurance Company,Health Insurance Issuer
2437,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2437,Cigna Health Life Insurance Company,Health Insurance Issuer
2438,in-network file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2438,Cigna Health Life Insurance Company,Health Insurance Issuer


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,Cigna Health Life Insurance Company,Health Insurance Issuer
1,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,1,Cigna Health Life Insurance Company,Health Insurance Issuer
2,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,2,Cigna Health Life Insurance Company,Health Insurance Issuer
3,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,3,Cigna Health Life Insurance Company,Health Insurance Issuer
4,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,4,Cigna Health Life Insurance Company,Health Insurance Issuer
...,...,...,...,...,...
18185,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,18185,Cigna Health Life Insurance Company,Health Insurance Issuer
18186,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,18186,Cigna Health Life Insurance Company,Health Insurance Issuer
18187,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,18187,Cigna Health Life Insurance Company,Health Insurance Issuer
18188,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,18188,Cigna Health Life Insurance Company,Health Insurance Issuer


,plan_name,plan_id_type,plan_id,plan_market_type,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,Aetna Bronze PPO5900 50/50 HSA OffMarketplace,HIOS,53357ME0040055,group,0,Aetna Life insurance Company,Health Insurance Issuer
1,Aetna Silver PPO 5500 70/50 Off Marketplace,HIOS,53357ME0040060,group,0,Aetna Life insurance Company,Health Insurance Issuer
2,Aetna Gold PPO 2500 70/50 Off Marketplace,HIOS,53357ME0040057,group,0,Aetna Life insurance Company,Health Insurance Issuer
3,Aetna Silver OAEPO 5500 80% PY,HIOS,39159CT0140001,group,1,Aetna Life insurance Company,Health Insurance Issuer
4,Aetna Gold PPO 750 70/50,HIOS,11082AK0060032,group,2,Aetna Life insurance Company,Health Insurance Issuer
...,...,...,...,...,...,...,...
815,Aetna OOS Broad PPO Gold 3000 80/50,HIOS,84251AZ0100190,group,28,Aetna Life insurance Company,Health Insurance Issuer
816,Aetna OOS Broad PPO Gold 3500 80/50,HIOS,84251AZ0100175,group,28,Aetna Life insurance Company,Health Insurance Issuer
817,Aetna OOS Broad PPO Silver 4000 70/50,HIOS,84251AZ0100176,group,28,Aetna Life insurance Company,Health Insurance Issuer
818,Aetna OOS Broad PPO Bronze 5700 70/50 HSA,HIOS,84251AZ0100185,group,28,Aetna Life insurance Company,Health Insurance Issuer


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,0,Aetna Life insurance Company,Health Insurance Issuer
1,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,1,Aetna Life insurance Company,Health Insurance Issuer
2,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,2,Aetna Life insurance Company,Health Insurance Issuer
3,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,3,Aetna Life insurance Company,Health Insurance Issuer
4,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,4,Aetna Life insurance Company,Health Insurance Issuer
5,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,5,Aetna Life insurance Company,Health Insurance Issuer
6,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,6,Aetna Life insurance Company,Health Insurance Issuer
7,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,7,Aetna Life insurance Company,Health Insurance Issuer
8,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,8,Aetna Life insurance Company,Health Insurance Issuer
9,in-network file,https://mrf.healthsparq.com/aetnacvs-egress.no...,9,Aetna Life insurance Company,Health Insurance Issuer


,description,location,reporting_structure_number,reporting_entity_name,reporting_entity_type
0,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,0,Aetna Life insurance Company,Health Insurance Issuer
1,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,1,Aetna Life insurance Company,Health Insurance Issuer
2,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,2,Aetna Life insurance Company,Health Insurance Issuer
3,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,3,Aetna Life insurance Company,Health Insurance Issuer
4,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,4,Aetna Life insurance Company,Health Insurance Issuer
5,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,5,Aetna Life insurance Company,Health Insurance Issuer
6,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,6,Aetna Life insurance Company,Health Insurance Issuer
7,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,7,Aetna Life insurance Company,Health Insurance Issuer
8,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,8,Aetna Life insurance Company,Health Insurance Issuer
9,allowed amount file,https://mrf.healthsparq.com/aetnacvs-egress.no...,9,Aetna Life insurance Company,Health Insurance Issuer


ValueError: Expecting a .json or .json.gz file...